# Bayesian optimization of architecture parameters

**Idea:** given an optimization task and an architecture, find the best set of parameters in a given set.

**Why Bayesian?** Because it is an adaptive optimization procedure, that starts sampling the parameters to try from some initial distributions (priors) provided by the user and then uses experience to update the sampling distribution, in order to maximize the expected performance on the task.

Notation:
- $\overline{\theta} = (\theta_1, \ldots, \theta_m)$ is a set of parameters;
- $\theta_{-i}$ is a set of parameters excluding $\theta_i$;
- $\theta_{ij}$ is the j-th possible value of the i-th parameter; 
- $V(\overline{\theta};\lambda)$ is the value (performance) achieved in the task from that set of parameters and learning rate $\lambda$;
- $V(\overline{\theta})$ is the average value achieved by a set of parameters for different learning rates;
- $V(\overline{\theta})=\sum_{i=1}^m v_i(\theta_i)$ is the value decomposition that for semplicity we assume (independent contribution from different parameters, neglecting interactions between them);

I estimate $v_{ij} = v_i(\theta_{ij})$ as follows:
$$v_{ij} = \underset{\{\theta_{-i}\}}E[V(\theta_{-i},\theta_{ij}] - \underset{\{\overline{\theta}\}}E[V(\overline{\theta})]$$
Basically is the difference between the expected value marginal to having selected $\theta_i = \theta_{ij}$ w.r.t. the global expected value.

I define a prior $\Pi_i = (\pi_1, \ldots, \pi_{n_i})$ for each parameter $\theta_i$ and a number of samples $N$ associated to that prior (e.g. $N=2$); this basically tells me how much my prior will weight in the final probability distribution.

**Probability distribution update:** After we try out a particular parameter configuration, we want to update the probability of sampling each parameter using the information that we gained (i.e. the value achieved by this particular configuration). To do so, first we compute the values $v_{ij}$ with the formula introduced above, then we compute the advantage in choosing $v_{ij}$ as $a_{ij} = \frac{v_{ij} - \overline{v_i}}{\overline{v_i}}$; Now we compute the biased advantages $a^{(b)}_{ij}$ as a weighted sum between the prior and the real advantage, with weights $N$ and $f_{ij}$ and finally we take a softmax to get the probability distribution $p_{ij}$.

Actually here the greatest problem is that the advantages and the prior have different scales at the moment, which means that our update won't be as effective as it should. The first thing that we need to solve is that we choose the prior using our intuition of what those probabilities mean, but we need to convert it in relative advantages if we want to compare it with the real advantages that we computed. One way of doing so is to consider the prior as a result of a softmax and to invert the operation:
$$\pi_{ij} = \frac{exp(q_{ij})}{\sum_k exp(q_{ik})}$$
This yields $q_{ij} = log(\pi_{ij}) + C_i$, with $C_i$ a constant that we can choose. Since the advantages $a_{ij}$ have zero average, we can choose $C_i$ so that even the $q_{ij}$ have zero average. In other words we cannot decide the range of values that $q_{ij}$ span, but only the center of that interval.
So $C_i = - \frac{1}{n_i}\sum_k log(\pi_{ik})$ and finally $$q_{ij} = log(\pi_{ij}) - \frac{1}{n_i}\sum_k log(\pi_{ik})$$
So then 
$$a^{(b)}_{ij} = \frac{Nq_{ij}+f_{ij}a_{ij}}{N+f_{ij}}$$
and 
$$p_{ij} = \frac{exp(a^{(b)}_{ij})}{\sum_k exp(a^{(b)}_{ik})}$$

In [23]:
class BayesHPTuning():
    
    def __init__(self, value_priors_dict, evaluation_instance, N=2, lambdas=[5e-3,1e-3,5e-4]):
        
        self.params = {}
        for param in value_priors_dict.keys():
            self.params[param] = BayesParam(*value_priors_dict[param], prior_sample_weight=N)
            
        self.eval = evaluation_instance
        self.lambdas = lambdas
        self.history = []
    
    def step(self):
        start = time.time()
        
        HPs, HPs_probs = self.sample_params()
        print("\n"+"="*40)
        print("Configuration sampled: ")
        for p in HPs:
            print('\t'+p+' : ',HPs[p], ' - prob: %.2f'%HPs_probs[p])
        print("="*40)
        V, V_lambda = self.evaluate_params(HPs)
        self.update_params(V, V_lambda)
        
        elapsed = time.time() - start
        
        # here can be implemented conditions for logging, saving and stopping the whole thing
        self.print_step(HPs, HPs_probs, V, V_lambda, elapsed)
        
        return
    
    def sample_params(self):
        HPs = {}
        HPs_probs = {}
        for param in self.params:
            value, prob = self.params[param].sample()
            HPs[param] = value
            HPs_probs[param] = prob
        return HPs, HPs_probs
    
    def evaluate_params(self, HPs):
        # both of shape (len(lambdas), n_epochs)
        train_V_lambda, val_V_lambda = self.eval.evaluate_params(HPs, self.lambdas)
        
        d = dict(HPs=HPs, lambdas=self.lambdas, train_V_lambda=train_V_lambda, val_V_lambda=val_V_lambda)
        self.history.append(d)
        
        V = val_V_lambda.mean()
        V_lambda = val_V_lambda.mean(axis=1)
        
        return V, V_lambda
    
    def update_params(self, V, V_lambda):
        for param in self.params:
            self.params[param].update_stat(V, V_lambda)
        return
    
    def print_step(self, HPs, HPs_probs, V, V_lambda, elapsed):
        print("\n"+"="*40)
        print("Configuration sampled: ")
        for p in HPs:
            print('\t'+p+' : ',HPs[p], ' - prob: %.2f'%HPs_probs[p])
        print("="*40)
        for i, l in enumerate(self.lambdas):
            print("lambda: %.4f - V: %4f"%(self.lambdas[i],V_lambda[i]))
        print("Average V: %4f"%V)
        print("Time elapsed: %.2f s"%(elapsed))
        return
    
    def return_best_model(self):
        HPs = {}
        for param in self.params:
            probs = self.params[param].get_updated_sampling_probs()
            idx = np.argmax(probs)
            v = self.params[param].values[idx]
            HPs[param] = v
        return HPs
    
    def return_param_probs(self, param_name):
        values = self.params[param_name].values
        probs = self.params[param_name].get_updated_sampling_probs()
        return values, probs

In [24]:
class BayesParam():
    def __init__(self, values, priors, prior_sample_weight):
        self.values = values
        self.priors = priors
        self.N = prior_sample_weight
        self.global_V = []
        self.last_sampled = None
        self.stat = {}
        for idx in range(len(values)):
            self.stat[idx] = {'V_lambda':[], 'V':[], 'freq':0}
            
    def sample(self):
        probs = self.get_updated_sampling_probs()
        idx = np.random.choice(np.arange(len(self.values)), p=probs)
        value = self.values[idx]
        self.last_sampled = idx
        return value
        
    def get_updated_sampling_probs(self):
        expected_global_V = np.mean(self.global_V)
        advantages = []
        for idx in self.stat:
            if self.stat['freq'] != 0:
                expected_Vj = np.mean(self.stat[i]['V'])
                adv_j = expected_Vj - expected_global_V
            else:
                adv_j = 0 # every value would do
            biased_adv_j = (self.N*self.priors[idx]+self.stat['freq']*adv_j)/(self.N+self.stat['freq'])
            advantages.append(biased_adv_j)
        # sampling probs are the softmax of the biased advantages 
        advantages = np.array(advantages)
        probs = np.exp(advantages)/np.exp(advantages).sum()
        return probs
    
    def update_stat(self, V, V_lambda):
        idx = self.last_sampled
        self.stat[idx]['V'].append(V)
        self.stat[idx]['V_lambda'].append(V_lambda)
        self.stat[idx]['freq'] += 1
        return